In [2]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer

In [3]:
dataset = pd.read_csv('data/keywords_.csv', sep=';')

In [4]:
df = pd.DataFrame(dataset)
df

,'AdGroupId','CampaignId','Id','Keyword','ServingStatus','State','Status','StrategyPriority'
0,3767436593,42244009,16348493015,'фас тарифы -жкх','ELIGIBLE','ON','ACCEPTED','NORMAL'
1,3767436593,42244009,16348696779,'фас тарифы жкх','ELIGIBLE','ON','ACCEPTED','NORMAL'
2,3767470344,42244009,16348696780,'определение суд экспертизы','ELIGIBLE','ON','ACCEPTED','NORMAL'
3,3767470344,42244009,16348696782,'определение суда +о назначении экспертизы','ELIGIBLE','ON','ACCEPTED','NORMAL'
4,3767470345,42244009,16348696783,'жкх последние изменения','ELIGIBLE','ON','ACCEPTED','NORMAL'
...,...,...,...,...,...,...,...,...
3995,3126857265,26790451,13056964783,'DC Shoes Banshee','ELIGIBLE','ON','ACCEPTED','NORMAL'
3996,3126857265,26790451,13056964790,'Shoes Banshee -dc','ELIGIBLE','ON','ACCEPTED','NORMAL'
3997,3126857265,26790451,13056964798,'686 Mannual Standard -w','ELIGIBLE','ON','ACCEPTED','NORMAL'
3998,3126857265,26790451,13056964806,'686 Mannual Standard W','ELIGIBLE','ON','ACCEPTED','NORMAL'


In [26]:
df['\'Keyword\''] = list(map(lambda x: re.sub(r'\s-[a-zA-Zа-яА-Я!0-9]+', '', x), df['\'Keyword\'']))

In [27]:
df

,'AdGroupId','CampaignId','Id','Keyword','ServingStatus','State','Status','StrategyPriority'
0,3767436593,42244009,16348493015,'фас тарифы','ELIGIBLE','ON','ACCEPTED','NORMAL'
1,3767436593,42244009,16348696779,'фас тарифы жкх','ELIGIBLE','ON','ACCEPTED','NORMAL'
2,3767470344,42244009,16348696780,'определение суд экспертизы','ELIGIBLE','ON','ACCEPTED','NORMAL'
3,3767470344,42244009,16348696782,'определение суда +о назначении экспертизы','ELIGIBLE','ON','ACCEPTED','NORMAL'
4,3767470345,42244009,16348696783,'жкх последние изменения','ELIGIBLE','ON','ACCEPTED','NORMAL'
...,...,...,...,...,...,...,...,...
3995,3126857265,26790451,13056964783,'DC Shoes Banshee','ELIGIBLE','ON','ACCEPTED','NORMAL'
3996,3126857265,26790451,13056964790,'Shoes Banshee','ELIGIBLE','ON','ACCEPTED','NORMAL'
3997,3126857265,26790451,13056964798,'686 Mannual Standard','ELIGIBLE','ON','ACCEPTED','NORMAL'
3998,3126857265,26790451,13056964806,'686 Mannual Standard W','ELIGIBLE','ON','ACCEPTED','NORMAL'


In [35]:
df_result = pd.DataFrame({'Keyword_x' : [], 'AdGroupId_x' : [], 'Keyword_y' : [], 'AdGroupId_y' : [], 'crossed' : []})

In [36]:
frame = list()
tokenizer = RegexpTokenizer(r'[a-zA-Zа-яА-Я+!0-9]+')
for i in df[:][:1000].index:
    agi = df['\'AdGroupId\''][i]
    series = df.where(df['\'AdGroupId\''] != agi)
    series.dropna(inplace=True)
    for s in series.index:
        word_x = df['\'Keyword\''][i]
        word_y = df['\'Keyword\''][s]
        kws_x = tokenizer.tokenize(word_x.lower())
        kws_y = tokenizer.tokenize(word_y.lower())
        agiy = df['\'AdGroupId\''][s]
        subset = [word for word in kws_x if word.lower() in kws_y]
        if len(subset) > 1:
            subset = ', '.join(map(str, subset))
            df_result = pd.concat([df_result, pd.DataFrame({'Keyword_x' : [word_x], 'AdGroupId_x' : [agi], 
                                                            'Keyword_y' : [word_y], 'AdGroupId_y' : [agiy], 
                                                            'crossed' : [subset]})])
            #frame.append(pd.DataFrame({'Keyword_x' : [word_x], 'AdGroupId_x' : [agi], 'Keyword_y' : [word_y], 
            #                           'AdGroupId_y' : [agiy], 'crossed' : [subset]}))

KeyboardInterrupt: 

In [37]:
#df_result = pd.concat(frame)
df_result[['AdGroupId_x', 'AdGroupId_y']] = df_result[['AdGroupId_x', 'AdGroupId_y']].astype('int64')

In [38]:
df_result.to_csv(r'data/keywords_ext.csv', index=None)

In [39]:
df_result

,Keyword_x,AdGroupId_x,Keyword_y,AdGroupId_y,crossed
0,'определение суд экспертизы',3767470344,'суд экспертизы рф',3767470352,"суд, экспертизы"
0,'изменения +в жкх',3767470345,'жкх подали +в суд',3767470351,"+в, жкх"
0,'оплата жкх',3767470346,'оплата коммунальных услуг жкх',3767470348,"оплата, жкх"
0,'стоимость услуг жкх',3767470346,'оплата коммунальных услуг жкх',3767470348,"услуг, жкх"
0,'стоимость услуг жкх',3767470346,'аудиторская компания стоимость услуг',3767585626,"стоимость, услуг"
...,...,...,...,...,...
0,'купить участок +в полазне',1718324047,'купить квартиру студию +в перми',3870781201,"купить, +в"
0,'купить участок +в полазне',1718324047,'купить квартиру студию +в ипотеку',3870781201,"купить, +в"
0,'купить участок +в полазне',1718324047,'купить квартиру +в новостройке +с отделкой',3870781202,"купить, +в"
0,'купить участок +в полазне',1718324047,'купить 1 комнатную квартиру +в новостройке не...,3870781204,"купить, +в"
